In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
!pip install mtcnn
!pip install opencv-python

# This will list the files in 'My Drive', including the shortcuts.
drive_root = '/content/drive/MyDrive/'
os.listdir(drive_root)


to get frames for original & manipulated and perfrom mtcnn

In [ ]:
import os
import cv2
import time
from mtcnn import MTCNN

detector = MTCNN()
log_file = "processed_frames.log"

# Load already processed paths
processed = set()
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        processed = set(line.strip() for line in f.readlines())

def process_frame(frame_path):
    if frame_path in processed:
        return False  # Not newly processed

    try:
        image = cv2.imread(frame_path)
        if image is None:
            return False

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(rgb_image)

        for face in faces:
            confidence = face['confidence']
            box = face['box']
            keypoints = face['keypoints']

            if confidence < 0.90 or box[2] < 50 or box[3] < 50:
                continue
            if not all(k in keypoints for k in ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']):
                continue
            if abs(keypoints['left_eye'][0] - keypoints['right_eye'][0]) < 15:
                continue

            x, y, w, h = box
            x, y = max(0, x), max(0, y)
            cropped_face = image[y:y+h, x:x+w]

            cv2.rectangle(cropped_face, (0, 0), (w, h), (0, 255, 0), 2)
            cv2.imwrite(frame_path, cropped_face)

            with open(log_file, 'a') as logf:
                logf.write(frame_path + '\n')
            processed.add(frame_path)

            return True  # Newly processed

    except Exception as e:
        print(f"💥 Error processing {frame_path}: {e}")

    time.sleep(0.05)
    return False

def process_all_images(directory_path, max_folders=100):
    folders_processed = 0

    for folder_name in sorted(os.listdir(directory_path)):
        if folders_processed >= max_folders:
            break

        folder_path = os.path.join(directory_path, folder_name)
        if os.path.isdir(folder_path):
            jpg_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
            newly_processed_in_folder = 0
            total_in_folder = len(jpg_files)

            for file_name in jpg_files:
                file_path = os.path.join(folder_path, file_name)
                if process_frame(file_path):
                    newly_processed_in_folder += 1

            print(f"📂 Done with folder {folder_name}: {total_in_folder} total, {newly_processed_in_folder} newly processed")
            folders_processed += 1

if __name__ == "__main__":
    directory_path = '/content/drive/MyDrive/FaceForensicsData/frames_for_original_videos'
    process_all_images(directory_path, max_folders=100)
    print(f"🎉 Done! Processed up to 100 folders.")


get the labels

put them into v1 and v2 inception resnet

In [ ]:
import torch
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Transformation
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Base paths (Limit to only one video from each category)
data_sources = {
    "/content/drive/MyDrive/FaceForensicsData/frames_for_original_videos": 0,   # Real
    "/content/drive/MyDrive/FaceForensicsData/frames_for_FaceSwap": 1 # Fake
}

# Hold all embeddings and labels
all_embeddings = []
all_labels = []

# Process only the first video from both real and fake datasets
for folder_path, label in data_sources.items():
    # Select the first video folder from the directory (only one video folder)
    video_folders = sorted(os.listdir(folder_path))[:1]  # Limit to the first folder

    for video_folder in tqdm(video_folders, desc=f"Processing {folder_path}"):
        video_path = os.path.join(folder_path, video_folder)
        if not os.path.isdir(video_path):
            continue

        # Process all frames (100 frames) inside this folder
        for img_name in sorted(os.listdir(video_path)):
            if not img_name.endswith('.jpg'):
                continue

            img_path = os.path.join(video_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tensor = transform(img_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                embedding = model(img_tensor).cpu().numpy().flatten()  # shape: (512,)

            # Append the embedding and label to lists
            all_embeddings.append(embedding)
            all_labels.append(label)

# Convert to arrays
X = np.array(all_embeddings)  # shape: (num_images, 512)
y = np.array(all_labels)      # shape: (num_images,)

# Save embeddings and labels to Google Drive
save_path = '/content/drive/MyDrive/embeddings_and_labels/'

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save embeddings and labels to Google Drive
np.save(os.path.join(save_path, "test_embeddings.npy"), X)
np.save(os.path.join(save_path, "test_labels.npy"), y)

print(f"✅ Finished processing. Embeddings shape: {X.shape}, Labels shape: {y.shape}")


get the labels and save them

In [ ]:
import torch
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm
from itertools import islice  # Import for limiting folder count

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Transformation
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Base paths
data_sources = {
    "/content/drive/MyDrive/FaceForensicsData/frames_for_original_videos": 0,   # Real
    "/content/drive/MyDrive/FaceForensicsData/frames_for_FaceSwap": 1 # Fake
}

# Hold all embeddings and labels
all_embeddings = []
all_labels = []

# Process only the first 100 folders from both real and fake datasets
for folder_path, label in data_sources.items():
    # Use islice to limit the number of folders processed (100 folders from each category)
    video_folders = list(islice(sorted(os.listdir(folder_path)), 100))  # Limit to first 100 folders

    for video_folder in tqdm(video_folders, desc=f"Processing {folder_path}"):
        video_path = os.path.join(folder_path, video_folder)
        if not os.path.isdir(video_path):
            continue

        # Process all frames (100 frames) inside each folder
        for img_name in sorted(os.listdir(video_path)):
            if not img_name.endswith('.jpg'):
                continue

            img_path = os.path.join(video_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tensor = transform(img_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                embedding = model(img_tensor).cpu().numpy().flatten()  # shape: (512,)

            # Append the embedding and label to lists
            all_embeddings.append(embedding)
            all_labels.append(label)

# Convert to arrays
X = np.array(all_embeddings)  # shape: (num_images, 512)
y = np.array(all_labels)      # shape: (num_images,)

# Save embeddings and labels to Google Drive
save_path = '/content/drive/MyDrive/embeddings_and_labels/'

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save embeddings and labels to Google Drive
np.save(os.path.join(save_path, "all_embeddings.npy"), X)
np.save(os.path.join(save_path, "all_labels.npy"), y)

print(f"✅ Finished processing. Embeddings shape: {X.shape}, Labels shape: {y.shape}")


In [ ]:
for v2


In [ ]:
import torch
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm
from itertools import islice  # Import for limiting folder count

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Transformation
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Base paths
data_sources = {
    "/content/drive/MyDrive/FaceForensicsData/frames_for_original_videos": 0,   # Real
    "/content/drive/MyDrive/FaceForensicsData/frames_for_FaceSwap": 1 # Fake
}

# Hold all embeddings and labels
all_embeddings = []
all_labels = []

# Process only the first 100 folders from both real and fake datasets
for folder_path, label in data_sources.items():
    # Use islice to limit the number of folders processed (100 folders from each category)
    video_folders = list(islice(sorted(os.listdir(folder_path)), 100))  # Limit to first 100 folders

    for video_folder in tqdm(video_folders, desc=f"Processing {folder_path}"):
        video_path = os.path.join(folder_path, video_folder)
        if not os.path.isdir(video_path):
            continue

        # Process all frames (100 frames) inside each folder
        for img_name in sorted(os.listdir(video_path)):
            if not img_name.endswith('.jpg'):
                continue

            img_path = os.path.join(video_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tensor = transform(img_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                embedding = model(img_tensor).cpu().numpy().flatten()  # shape: (512,)

            # Append the embedding and label to lists
            all_embeddings.append(embedding)
            all_labels.append(label)

# Convert to arrays
X = np.array(all_embeddings)  # shape: (num_images, 512)
y = np.array(all_labels)      # shape: (num_images,)

# Save embeddings and labels to Google Drive
save_path = '/content/drive/MyDrive/embeddings_and_labels/'

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save embeddings and labels to Google Drive as v2 versions
np.save(os.path.join(save_path, "v2_embeddings.npy"), X)
np.save(os.path.join(save_path, "v2_labels.npy"), y)

print(f"✅ Finished processing. Embeddings shape: {X.shape}, Labels shape: {y.shape}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from models.mem_task2_model import BiLSTMAttentionModel

class BiLSTMAttentionModel(nn.Module):
    def __init__(self, input_dim=1024, num_classes=2):
        super(BiLSTMAttentionModel, self).__init__()

        self.feature_enhancer = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # First BiLSTM layer (128 hidden units, bidirectional)
        self.bilstm1 = nn.LSTM(256, 128, batch_first=True, bidirectional=True)

        # Second BiLSTM layer (64 hidden units, bidirectional)
        self.bilstm2 = nn.LSTM(128 * 2, 64, batch_first=True, bidirectional=True)

        # Attention mechanism over final BiLSTM output
        self.attention = nn.Linear(64 * 2, 1)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(64 * 2, num_classes)
        )

    def forward(self, x):
        # x: [B, T, 1024]
        B, T, D = x.shape

        x = self.feature_enhancer(x)           # [B, T, 256]
        x, _ = self.bilstm1(x)                 # [B, T, 256]
        x, _ = self.bilstm2(x)                 # [B, T, 128]

        attn_weights = F.softmax(self.attention(x), dim=1)  # [B, T, 1]
        context = torch.sum(attn_weights * x, dim=1)        # [B, 128]

        out = self.classifier(context)         # [B, 2]
        return out


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from models.mem_task2_model import BiLSTMAttentionModel

# ===== Load and preprocess data =====
all_features = np.load("/content/drive/MyDrive/deepfake_detection/data/combined_embeddings.npy")
all_labels = np.load("/content/drive/MyDrive/deepfake_detection/data/combined_labels.npy")

# Trim to multiple of 8
total_frames = (len(all_features) // 8) * 8
all_features = all_features[:total_frames]
all_labels = all_labels[:total_frames]

# Reshape and prepare labels
all_features = all_features.reshape(-1, 8, 1024)
all_labels = all_labels.reshape(-1, 8)
majority_labels = np.array([np.bincount(seq).argmax() for seq in all_labels])

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(all_features, majority_labels, test_size=0.2, stratify=majority_labels, random_state=42)
np.save("/content/drive/MyDrive/deepfake_detection/data/test_features.npy", X_test)
np.save("/content/drive/MyDrive/deepfake_detection/data/test_labels.npy", y_test)

# Prepare training data (frame-level to sequence)
train_features = X_train.reshape(-1, 1024)
train_labels = np.repeat(y_train, 8)

# ===== Dataset class =====
class FeatureSequenceDataset(Dataset):
    def __init__(self, features, labels, sequence_length=8):
        self.features = features
        self.labels = labels
        self.sequence_length = sequence_length
        self.num_sequences = len(self.features) // sequence_length

    def __len__(self):
        return self.num_sequences

    def __getitem__(self, idx):
        start = idx * self.sequence_length
        end = start + self.sequence_length
        sequence = self.features[start:end]
        label_seq = self.labels[start:end]
        label = np.bincount(label_seq).argmax()
        return torch.tensor(sequence, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Load dataset
train_dataset = FeatureSequenceDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ===== Model, Loss, Optimizer =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttentionModel(input_dim=1024, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# ===== Training loop =====
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# ===== Save trained model =====
torch.save(model.state_dict(), "/content/drive/MyDrive/deepfake_detection/mem_task2_bilstm_adam.pth")
print("✅ Model trained using Adam and saved successfully!")

In [ ]:
import torch
import numpy as np
import pandas as pd
from models.mem_task2_model import BiLSTMAttentionModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ===== Load test data =====
features = np.load("/content/drive/MyDrive/deepfake_detection/data/test_features.npy")   # shape: [N, 8, 1024]
labels = np.load("/content/drive/MyDrive/deepfake_detection/data/test_labels.npy")       # shape: [N]

# ===== Load trained model from Adam training =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttentionModel(input_dim=1024, num_classes=2).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/deepfake_detection/mem_task2_bilstm_adam.pth", map_location=device))
model.eval()

print("✅ Adam-trained model loaded. Starting evaluation on test set...")

# ===== Run inference =====
video_preds = []
for seq in features:
    input_tensor = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(device)  # shape: [1, 8, 1024]
    with torch.no_grad():
        output = model(input_tensor)                         # shape: [1, 2]
        probs = torch.softmax(output, dim=1)                 # softmax over output
        pred = torch.argmax(probs, dim=1).item()             # final predicted class
        video_preds.append(pred)

# ===== Evaluate =====
acc = accuracy_score(labels, video_preds)
prec = precision_score(labels, video_preds)
rec = recall_score(labels, video_preds)
f1 = f1_score(labels, video_preds)

print("\n🎯 Adam Model Evaluation Results:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# ===== Save results to CSV =====
df = pd.DataFrame({
    "Video_Index": list(range(len(labels))),
    "Predicted_Label": video_preds,
    "True_Label": labels.tolist()
})
df.to_csv("/content/drive/MyDrive/deepfake_detection/task3_video_results_adam.csv", index=False)
print("📁 Adam video-level predictions saved to task3_video_results_adam.csv")
